In [1]:
import ast
import dask.dataframe as dd
import glob
import json
import multiprocessing as mp
import nltk
import numpy as np
import pandas as pd
import spacy
import string
import textacy

from collections import Counter, defaultdict
from gensim.models.word2vec import Word2Vec
from IPython.core.debugger import Pdb
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
nlp = spacy.load('en_core_web_md')
sw = stopwords.words('english')

%matplotlib inline

In [3]:
wv = Word2Vec.load('./w2v_reviews_v2.model').wv

In [4]:
pool = mp.Pool(processes=mp.cpu_count())

In [5]:
df = pd.read_csv('./Electronics_5.csv', encoding = 'utf8')
meta_df = pd.read_csv('./meta_electronics.csv', encoding = 'utf8')

In [6]:
test_df = df[df.asin == 'B003ELYQGG']
test_meta = meta_df[meta_df.asin == 'B003ELYQGG']

del df
del meta_df

In [7]:
docs = list(nlp.pipe(test_df.reviewText, n_threads=8, batch_size=100, entity=False))

In [8]:
sents = []
for doc in docs:
    sents.extend(list(doc.sents))
sent_docs = list(nlp.pipe([s.text for s in sents], n_threads=8))

8.3184808655262

In [69]:
noun_chunks = Counter()

for sent in sent_docs:
    noun_chunks.update([' '.join([w.lower_ for w in nc if w.tag_ == 'NN']) for nc in sent.noun_chunks])

list(sent.sents)[0].root

listen

In [64]:
test_df.reviewText[test_df.reviewText.str.contains('my ears')].values

array([ 'Not terrible for the price but I will agree that that the bass is lacking, basically just a high tremble.  I didn\'t realize it but I had a pair of these before that I found at a store a year or so ago.  They still work so I can\'t complain, other headphones I had lasted for only a few months before they stopped working.  So I will give the product some points for "going strong."If your not too picky about sound quality than these are perfect for the price.  They have three different sized "buds" so you can choose the right size for your ear.  I have to use the large size even though I have small ears to get good sound quality.  However they do bother the inside of my ears due to the way they are designed, as the plastic seems to rest on the outer walls of my ear.',
       'Good sounding and fitting ear buds. I have bought numerous ear/head phones over the years and these sound very good for the money. Why spend hundreds?They fit my ears and my kids ears.',
       "OK, I have 

In [70]:
noun_chunks.most_common(50)

[('', 26538),
 ('ear', 1341),
 ('sound', 1086),
 ('price', 1071),
 ('quality', 858),
 ('pair', 705),
 ('bass', 517),
 ('music', 332),
 ('set', 232),
 ('fit', 215),
 ('time', 205),
 ('product', 192),
 ('cord', 191),
 ('noise', 190),
 ('money', 169),
 ('something', 155),
 ('gym', 131),
 ('work', 131),
 ('earbuds', 128),
 ('lot', 126),
 ('color', 122),
 ('use', 116),
 ('volume', 114),
 ('way', 111),
 ('case', 101),
 ('design', 98),
 ('size', 94),
 ('thing', 93),
 ('job', 89),
 ('value', 88),
 ('headphone', 86),
 ('audiophile', 86),
 ('end', 85),
 ('anything', 83),
 ('anyone', 79),
 ('couple', 73),
 ('rubber', 72),
 ('one', 72),
 ('shape', 72),
 ('wife', 72),
 ('side', 71),
 ('range', 70),
 ('problem', 70),
 ('head', 69),
 ('review', 68),
 ('deal', 64),
 ('year', 64),
 ('nothing', 64),
 ('comfort', 63),
 ('phone', 60)]

In [61]:
wv.wmdistance('The sound is good', 'This sounds great')

3.1755685653473957

In [37]:
wv.most_similar(positive=['boomy'], negative=['quality'])


[('overpowering', 0.5681169629096985),
 ('thump', 0.5253229737281799),
 ('punchy', 0.5065110921859741),
 ('overwhelming', 0.4652685523033142),
 ('muddy', 0.46367210149765015),
 ('booming', 0.4440876543521881),
 ('highs', 0.4227423071861267),
 ('harsh', 0.4116818606853485),
 ('vocals', 0.4080588221549988),
 ('jam', 0.3978205621242523)]

In [30]:
def parse_sentences(doc):
    return nlp(doc).sents        

In [36]:
sents = pool.map(nlp, test_df.reviewText)

PicklingError: Can't pickle <function BaseDefaults.<lambda> at 0x7fd78c9b8158>: attribute lookup BaseDefaults.<lambda> on spacy.language failed

In [3]:
wv.most_similar(['the', 'awful', 'sound'])

KeyError: "word 'the' not in vocabulary"